
Question 1

* In this dataset our desired target for classification task will be y variable - has the client subscribed a term   deposit or not.

In [1]:
import pandas as pd
import numpy as np

In [163]:
df = pd.read_csv(r"C:\Users\Aron\Desktop\ml-zoomcamp\03-classification\data\bank-full.csv", delimiter=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [164]:
features=['age', 'job', 'marital', 'education', 'balance', 'housing',
'contact', 'day', 'month', 'duration', 'campaign', 'pdays','previous', 'poutcome', 'y']

df = df[features]

In [180]:
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [154]:
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [158]:
df.isna().sum()


age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [156]:
df.education.mode()

0    secondary
Name: education, dtype: object

Question 2

In [159]:
correlation_matrix = df[numerical].corr()
correlation_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [126]:

print('age and balance:',round(correlation_matrix.iloc[0,1],3))
print('day and campaign:', round(correlation_matrix.iloc[2,4],3))
print('day and pdays:', round(correlation_matrix.iloc[2,5],3))
print('pdays and previous:', round(correlation_matrix.iloc[5,6],3))


age and balance: 0.098
day and campaign: 0.162
day and pdays: -0.093
pdays and previous: 0.455


Question 3

In [165]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})


In [78]:
from sklearn.model_selection import train_test_split

In [213]:
len(df)

45211

In [214]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [215]:
len(df_full_train), len(df_test)

(36168, 9043)

In [216]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [217]:
len(df_train), len(df_val), len(df_test)

(27126, 9042, 9043)

In [218]:
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [224]:
y_train=df_train.y.values
y_val=df_val.y.values
y_test=df_test.y.values

In [187]:
del df_train['y']
del df_val['y']
del df_test['y']

In [113]:
from sklearn.metrics import mutual_info_score

In [181]:
def calculate_mi(series):
    return mutual_info_score(series, df_train.y)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame('MI')


display(df_mi)

,MI
poutcome,0.029533
month,0.025090
contact,0.013356
housing,0.010343
job,0.007316
education,0.002697
marital,0.002050


Question 4

In [193]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [196]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [199]:
dv = DictVectorizer(sparse=False)

In [200]:
X_train = dv.fit_transform(train_dict)

In [243]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [203]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)


In [232]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [233]:
model.intercept_

array([-0.90516664])

In [234]:
model.coef_

array([[-1.86055010e-03,  1.11908456e-05, -8.80159934e-02,
         2.54937547e-01,  7.09884337e-02, -1.23109262e+00,
         6.45914549e-03,  4.11392149e-03, -4.19356359e-01,
        -2.44806988e-01, -5.44543331e-02, -1.86548961e-01,
        -8.62495210e-02, -8.18917120e-01,  7.28809179e-02,
        -2.46324187e-01, -2.40413200e-01, -2.81031666e-01,
        -8.06777342e-02,  3.32635266e-01, -2.71977266e-01,
        -1.54219271e-01,  2.45104220e-01, -1.62825337e-01,
         1.65375604e-02, -1.34855943e-01, -2.97314991e-01,
        -4.52990640e-01, -1.54861010e-01,  7.32609466e-02,
        -6.69306538e-01,  3.20313874e-01, -3.12993776e-01,
        -9.49618470e-01, -9.49881798e-01,  2.62805685e-01,
         1.24916454e+00, -4.79553242e-01, -9.00533969e-01,
         7.37167041e-01,  7.14009068e-01, -8.27759887e-04,
        -7.62264121e-01, -5.41203256e-01,  1.49196689e+00,
        -1.09366615e+00,  6.08358647e-03]])

In [235]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'duration',
       'education=primary', 'education=secondary', 'education=tertiary',
       'education=unknown', 'housing=no', 'housing=yes', 'job=admin.',
       'job=blue-collar', 'job=entrepreneur', 'job=housemaid',
       'job=management', 'job=retired', 'job=self-employed',
       'job=services', 'job=student', 'job=technician', 'job=unemployed',
       'job=unknown', 'marital=divorced', 'marital=married',
       'marital=single', 'month=apr', 'month=aug', 'month=dec',
       'month=feb', 'month=jan', 'month=jul', 'month=jun', 'month=mar',
       'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
       'poutcome=failure', 'poutcome=other', 'poutcome=success',
       'poutcome=unknown', 'previous'], dtype=object)

In [249]:
y_pred = model.predict_proba(X_val)[:,1]

In [250]:
y_pred

array([0.01252233, 0.01036543, 0.14732673, ..., 0.05617135, 0.00940853,
       0.28917677])

In [262]:
y = y_pred >= 0.5
round((y_val == y).mean(), 2)


np.float64(0.9)

Question 5

In [264]:
features = ['age','balance','marital','previous']

In [279]:
train_dict_small = df_train[features].to_dict(orient='records')
Val_dict_small = df_val[features].to_dict(orient='records')

In [270]:
dv_small = DictVectorizer(sparse=False)


In [274]:
X_small_train = dv_small.fit_transform(train_dict_small)

In [284]:
X_small_val = dv_small.transform(Val_dict_small)

In [277]:
model_small = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)


In [282]:
model_small.fit(X_small_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [285]:
y_pred = model_small.predict_proba(X_small_val)[:,1]

In [292]:
y = y_pred >= 0.5
original_accuracy = round((y_val == y).mean(), 2)
print('original_accuracy:', original_accuracy)

original_accuracy: 0.88


Question 6

In [309]:
C_values = [0.01, 0.1, 1, 10, 100]
best_accuracy = 0
best_C = None

for C in C_values:
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]  
    
    y = y_pred >= 0.5
    
    accuracy = round((y_val == y).mean(), 3)
    print(f'Accuracy for C={C}: {accuracy:.3f}')
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

print(f'Best C: {best_C} with accuracy: {best_accuracy:.3f}')


Accuracy for C=0.01: 0.902
Accuracy for C=0.1: 0.902
Accuracy for C=1: 0.902
Accuracy for C=10: 0.902
Accuracy for C=100: 0.902
Best C: 0.01 with accuracy: 0.902
